# Load data and concanate each word to one block

In [ ]:
 #load Raw data from TDT ECoG maschine, plz run ""
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import h5py
import scipy.io as scio
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
import matplotlib
plt.rcParams['axes.unicode_minus'] = False
# matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
# matplotlib.rcParams['font.size'] = 10
import matplotlib as mpl
mpl.rcParams['text.usetex'] = False
from matplotlib.colors import Normalize
from matplotlib.patches import Rectangle
import scipy.io as scio
import matplotlib.image as mpimg
from scipy import stats

freq = 400
HS = 45
elec = 74
PATH='/public/DATA/covert_reading/points'
bands_list = {
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 12),
    'beta': (12, 30),
    'gamma': (30, 70),
    'high gamma':(70,150),
    'else2':(150,min(freq/2, 300))
}

In [9]:
import numpy as np

block_data = np.load('/public/DATA/covert_reading/HS_preanalysis_ECoG/HS45/result/HS45_Block_overt_covert_1_4_400hz.npy', allow_pickle=True).item()
print(block_data.keys())

# Filter and concatenate covert data
covert_keys = [key for key in block_data.keys() if key.startswith('ECoG_covert_')]
covert_data = np.concatenate([block_data[key] for key in covert_keys], axis=0)

# Filter and concatenate overt data
overt_keys = [key for key in block_data.keys() if key.startswith('ECoG_overt_')]
overt_data = np.concatenate([block_data[key] for key in overt_keys], axis=0)
# Check the shapes of the concatenated data  timesteps = freq * time = 400 * (2.5 - (-1.5)) = 400 * 4 = 1600
print('Covert Data Shape:', covert_data.shape)
print('Overt Data Shape:', overt_data.shape)

# check the shapes of every key
print(f'overt_keys: {overt_keys}, length: len({len(overt_keys)})')

for key in overt_keys:
    print(f"The sampe of overt_data['{key}'] is {block_data[key].shape}")

print(f'covert_keys: {covert_keys}, length: len({len(covert_keys)})')

for key in covert_keys:
    print(f"The sampe of covert_data['{key}'] is {block_data[key].shape}")


dict_keys(['ECoG_sil', 'soundClipMat_sil', 'ECoG_finger_motor', 'soundClipMat_finger_motor', 'ECoG_overt_ba', 'soundClipMat_overt_ba', 'ECoG_overt_bu', 'soundClipMat_overt_bu', 'ECoG_overt_da', 'soundClipMat_overt_da', 'ECoG_overt_du', 'soundClipMat_overt_du', 'ECoG_overt_ga', 'soundClipMat_overt_ga', 'ECoG_overt_gu', 'soundClipMat_overt_gu', 'ECoG_covert_ba', 'soundClipMat_covert_ba', 'ECoG_covert_bu', 'soundClipMat_covert_bu', 'ECoG_covert_da', 'soundClipMat_covert_da', 'ECoG_covert_du', 'soundClipMat_covert_du', 'ECoG_covert_ga', 'soundClipMat_covert_ga', 'ECoG_covert_gu', 'soundClipMat_covert_gu'])
Covert Data Shape: (355, 256, 1600)
Overt Data Shape: (353, 256, 1600)
overt_keys: ['ECoG_overt_ba', 'ECoG_overt_bu', 'ECoG_overt_da', 'ECoG_overt_du', 'ECoG_overt_ga', 'ECoG_overt_gu'], length: len(6)
The sampe of overt_data['ECoG_overt_ba'] is (60, 256, 1600)
The sampe of overt_data['ECoG_overt_bu'] is (55, 256, 1600)
The sampe of overt_data['ECoG_overt_da'] is (63, 256, 1600)
The samp

# cut data to two tasks

In [2]:
def load_data(path, HS, freq, band):
    ''' 1. load data and concancate different words
        2. z-score for two task, baseline is (-0.2*onset : onset)    z-score is done, needn't to do it again
        3. cut data to [-0.2*onset, o.75*onset]
        4. return z_covert_data, z_overt_data  shape=(n_samples, n_electrodes, n_timesteps), timesteps = (0.75+0.2)*freq
    '''
    print(f'load data for HS{HS}, samplingrate is {freq}, band is {band}')
    lowfreq, highfreq = bands_list[band]
    block_data = np.load(os.path.join(path, f'HS{HS}/result/HS{HS}_Block_overt_covert_{int(lowfreq)}_{int(highfreq)}_{freq}hz.npy'), allow_pickle=True).item()
    # Filter and concatenate covert data
    covert_keys = [key for key in block_data.keys() if key.startswith('ECoG_covert_')]
    covert_data = np.concatenate([block_data[key] for key in covert_keys], axis=0)[:, :, int(1.3*freq):int((1.5+0.75)*freq)]

    # Filter and concatenate overt data
    overt_keys = [key for key in block_data.keys() if key.startswith('ECoG_overt_')]
    overt_data = np.concatenate([block_data[key] for key in overt_keys], axis=0)[:, :, int(1.3*freq):int((1.5+0.75)*freq)]

    # Check the shapes of the concatenated data
    print('Covert Data Shape:', covert_data.shape)
    print('Overt Data Shape:', overt_data.shape)

    return covert_data, overt_data

In [3]:
HS_list = [47,48,50,54,71,76,78,73]
freq_list = [400]
for HS in HS_list:
    for freq in freq_list:
        data_path = '/public/DATA/covert_reading/HS_preanalysis_ECoG/'
        save_path = f'/public/DATA/covert_reading/HS_preanalysis_ECoG/HS{HS}/aligned_data'
        os.makedirs(save_path, exist_ok=True)
        covert_data_all = {}
        overt_data_all = {}
        for band in bands_list.keys():
            covert_data, overt_data = load_data(data_path, HS, freq, band)
            covert_data_all[band] = covert_data
            overt_data_all[band] = overt_data

        np.save(os.path.join(save_path, f'covert_allbands_{freq}hz.npy'), covert_data_all)
        np.save(os.path.join(save_path, f'overt_allbands_{freq}hz.npy'), overt_data_all)

load data for HS47, samplingrate is 400, band is delta
Covert Data Shape: (345, 256, 380)
Overt Data Shape: (344, 256, 380)
load data for HS47, samplingrate is 400, band is theta
Covert Data Shape: (345, 256, 380)
Overt Data Shape: (344, 256, 380)
load data for HS47, samplingrate is 400, band is alpha
Covert Data Shape: (345, 256, 380)
Overt Data Shape: (344, 256, 380)
load data for HS47, samplingrate is 400, band is beta
Covert Data Shape: (345, 256, 380)
Overt Data Shape: (344, 256, 380)
load data for HS47, samplingrate is 400, band is gamma
Covert Data Shape: (345, 256, 380)
Overt Data Shape: (344, 256, 380)
load data for HS47, samplingrate is 400, band is high gamma
Covert Data Shape: (345, 256, 380)
Overt Data Shape: (344, 256, 380)
load data for HS47, samplingrate is 400, band is else2
Covert Data Shape: (345, 256, 380)
Overt Data Shape: (344, 256, 380)
load data for HS48, samplingrate is 400, band is delta
Covert Data Shape: (340, 256, 380)
Overt Data Shape: (340, 256, 380)
load